In [11]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf 
import tensorflow_hub as hub

from keras.layers import Input, Dense, Lambda, Dropout, BatchNormalization
from keras.regularizers import l2
from keras.models import Model
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

In [12]:
from model import USEClassifier

In [13]:
# module_url = "https://tfhub.dev/google/universal-sentence-encoder-lite/2"
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2"
#module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embed = hub.Module(module_url)

DEBUG:tensorflow:Initialize variable module_1/Embeddings_en/sharded_0:0 from checkpoint b'/var/folders/rh/hnmbdhn55f7833nysls18h4c0000gn/T/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables' with Embeddings_en/sharded_0
DEBUG:tensorflow:Initialize variable module_1/Embeddings_en/sharded_1:0 from checkpoint b'/var/folders/rh/hnmbdhn55f7833nysls18h4c0000gn/T/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables' with Embeddings_en/sharded_1
DEBUG:tensorflow:Initialize variable module_1/Embeddings_en/sharded_10:0 from checkpoint b'/var/folders/rh/hnmbdhn55f7833nysls18h4c0000gn/T/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables' with Embeddings_en/sharded_10
DEBUG:tensorflow:Initialize variable module_1/Embeddings_en/sharded_11:0 from checkpoint b'/var/folders/rh/hnmbdhn55f7833nysls18h4c0000gn/T/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables' with Embeddings_en/sharded_11
DEBUG:tensorflow:Ini

In [ ]:
messages = ['Indicies dipping, big names show an  bottom, see if they break or hold these lows #amzn #aapl #ba', 'Increased our stake in #aapl by 5%. Love the foldable idea, but don’t think people will pay $2,000 to be the first guinea pigs. #samsung #foldable']

In [ ]:
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(embed(messages))

    for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
        print("Message: {}".format(messages[i]))
        print("Embedding size: {}".format(len(message_embedding)))
        message_embedding_snippet = ", ".join(
            (str(x) for x in message_embedding))
        print("Embedding: [{}]\n".format(message_embedding_snippet))

In [ ]:
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), 
    	signature="default", as_dict=True)["default"]

In [ ]:
class USEClassifier:
    
    def __init__(self):
        self.embed_size = 512
        self.model = self.init_model()
        self.model_name = 'test'
        self.batch_size = 4
        
    def init_model(self):
        input_text = Input(shape=(1,), dtype=tf.string)
        embedding = Lambda(self._UniversalEmbedding, output_shape=(self.embed_size,))(input_text)
        dense = Dense(256, activation='relu', kernel_regularizer=l2(.01))(embedding)
        dropout = Dropout(.2)(dense)
        pred = Dense(3, activation='softmax', kernel_regularizer=l2(.005))(dropout)
        model = Model(inputs=[input_text], outputs=pred)
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['mse', 'acc'])
        model.summary()
        return model
    
    def train(self, train_X, train_y, test_X, test_y):
        callbacks = [EarlyStopping(monitor='val_loss', patience=2),
                TensorBoard(log_dir='logs/{}'.format(self.model_name), batch_size=self.batch_size, write_images=True),
                ModelCheckpoint(filepath=self.model_name + '.h5', save_best_only=True)]
        model.fit(train_X, 
                train_y,
                validation_data=(test_X, test_y),
                epochs=10,
                callbacks=callbacks,
                batch_size=32)
    
    def _Universalhttp://localhost:8888/notebooks/notebooks/Financial%20Sentiment.ipynb#Embedding(self, x):
        return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

In [14]:
data = pd.read_csv('../data/testv2.csv', header=None)
data.head()

,0,1
0,MargaretsBelly Amy Schumer is the stereotypica...,-1
1,danipitter I mean I get the hype around JLaw I...,-1
2,Amy Schumer at the GQmenoftheyear2015 party in...,-1
3,Amy Schumer may have brought us Trainwreck but...,-1
4,I just think that sports are stupid ampanyone ...,-1


In [15]:
from collections import Counter 
c = Counter()
for i in data.iloc[:,1].tolist():
    c[float(i)] += 1
c

Counter({-1.0: 11695, 0.0: 20162, 1.0: 8212})

In [16]:
train_X = data[0].tolist()
train_X = np.array(train_X, dtype=object)[:, np.newaxis]

labels =  data[1].tolist()
train_y = np.zeros(shape=(len(labels),3))
for index, val in enumerate(labels):
    if val == -1:
        train_y[index, 0] = 1
    elif val == 0:
        train_y[index, 1] = 1
    else:
        train_y[index, 2] = 1

In [17]:
test = pd.read_csv('../data/trainv2.csv', header=None)
test_X = test[0].tolist()
test_X = np.array(test_X, dtype=object)[:, np.newaxis]
test_y =  test[1].tolist()

labels =  test[1].tolist()
test_y = np.zeros(shape=(len(labels),3))
for index, val in enumerate(labels):
    if val == -1:
        test_y[index, 0] = 1
    elif val == 0:
        test_y[index, 1] = 1
    else:
        test_y[index, 2] = 1

In [18]:
train_X.shape

(40069, 1)

In [19]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [20]:
tf.logging.set_verbosity(2)
classifier = USEClassifier(epochs=10, batch_size=10, embed_size=512, model_name='test3', embedding=None)

DEBUG:tensorflow:Initialize variable module_2/Embeddings_en/sharded_0:0 from checkpoint b'/var/folders/rh/hnmbdhn55f7833nysls18h4c0000gn/T/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables' with Embeddings_en/sharded_0
DEBUG:tensorflow:Initialize variable module_2/Embeddings_en/sharded_1:0 from checkpoint b'/var/folders/rh/hnmbdhn55f7833nysls18h4c0000gn/T/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables' with Embeddings_en/sharded_1
DEBUG:tensorflow:Initialize variable module_2/Embeddings_en/sharded_10:0 from checkpoint b'/var/folders/rh/hnmbdhn55f7833nysls18h4c0000gn/T/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables' with Embeddings_en/sharded_10
DEBUG:tensorflow:Initialize variable module_2/Embeddings_en/sharded_11:0 from checkpoint b'/var/folders/rh/hnmbdhn55f7833nysls18h4c0000gn/T/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables' with Embeddings_en/sharded_11
DEBUG:tensorflow:Ini

In [22]:
# classifier.predict(train_X)
classifier.train(train_X=train_X, train_y=train_y, test_X=test_X, test_y=test_y)

Level 2:tensorflow:  _AggregatedGrads 2 x (?, 3) using add_n
Level 2:tensorflow:  _AggregatedGrads 2 x (256, 3) using add_n
Level 2:tensorflow:  _AggregatedGrads 2 x (?, 256) using add_n
Level 2:tensorflow:  _AggregatedGrads 2 x (256,) using add_n
Level 2:tensorflow:  _AggregatedGrads 2 x (256,) using add_n
Level 2:tensorflow:  _AggregatedGrads 2 x (256,) using add_n
Level 2:tensorflow:  _AggregatedGrads 2 x (256,) using add_n
Level 2:tensorflow:  _AggregatedGrads 4 x (?, 256) using add_n
Level 2:tensorflow:  _AggregatedGrads 2 x (512, 256) using add_n
Train on 40069 samples, validate on 14890 samples
Epoch 1/10


FailedPreconditionError: Table not initialized.
	 [[{{node lambda_2/module_2_apply_default/string_to_index_Lookup/hash_table_Lookup}} = LookupTableFindV2[Tin=DT_STRING, Tout=DT_INT64, _device="/job:localhost/replica:0/task:0/device:CPU:0"](module_2/string_to_index/hash_table, lambda_2/module_2_apply_default/compound_bigrams/boolean_mask/Gather, lambda_2/module_2_apply_default/string_to_index/hash_table/Const)]]

In [ ]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
#     callbacks = [EarlyStopping(monitor='val_loss', patience=2),
#                       TensorBoard(log_dir='logs/{}'.format(self.model_name), batch_size=self.batch_size, write_images=True),
#                       ModelCheckpoint(filepath='test.h5', save_best_only=True)]
    model.fit(train_X, 
            train_y,
            validation_data=(test_X, test_y),
            epochs=10,
            batch_size=32)
    model.save_weights('./model.h5')

In [ ]:
model.predict(np.array(['this stock is tanking really bad in the bear market']))

In [ ]:
from keras.models import load_model

In [ ]:
trained = model.load_weights('model.h5')

In [ ]:
with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    model.predict(np.array(['fuck', 'shit', 'bitch']))